In [1]:
# Initialize SSMs and LLMs
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
from tqdm import tqdm
import json

# general configs
DATASET_PATH = "alpaca.json"
TREE_DEPTH = 8 # tokens generated by SSM in one pass
MAX_GENERATION_LEN = 24 # max token to generate by LLM
CACHE_DIR = ".cache" # change to your dir
LOG_PATH = "logs/log_alpaca.txt"  # remember to mkdir logs
STATISTIC_PATH = "stats/stats_alpaca.txt" # remember to mkdir stats
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)
# TODO Find and set suitable generation configs for SSMs
ssm_generation_configs = [
    GenerationConfig(
        do_sample=True, temperature=0.5
    ),
    GenerationConfig(
        do_sample=True, temperature=0.5
    ),
    GenerationConfig(
        do_sample=True, temperature=0.5
    ),
    GenerationConfig(
        do_sample=True, temperature=0.5
    )
]

ssm_tokenizers = [
    AutoTokenizer.from_pretrained("JackFram/llama-68m", cache_dir=CACHE_DIR),
    AutoTokenizer.from_pretrained("JackFram/llama-160m", cache_dir=CACHE_DIR),
    AutoTokenizer.from_pretrained("facebook/opt-125m", cache_dir=CACHE_DIR),
    AutoTokenizer.from_pretrained("facebook/opt-125m", cache_dir=CACHE_DIR)
]

ssm_models = [
    AutoModelForCausalLM.from_pretrained("JackFram/llama-68m", cache_dir=CACHE_DIR).to(DEVICE),
    AutoModelForCausalLM.from_pretrained("JackFram/llama-160m", cache_dir=CACHE_DIR).to(DEVICE),
    AutoModelForCausalLM.from_pretrained("facebook/opt-125m", cache_dir=CACHE_DIR).to(DEVICE),
    AutoModelForCausalLM.from_pretrained("facebook/opt-125m", cache_dir=CACHE_DIR).to(DEVICE)
]

# TODO Find and set suitable generation config for LLM
llm_generation_config = GenerationConfig(do_sample=True, temperature=0.1)
llm_tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b", cache_dir=CACHE_DIR)
llm_model = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b", cache_dir=CACHE_DIR).to(DEVICE)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
"""
write the output to a file
"""
def log_print(*args, file_path=LOG_PATH, sep=' ', end='\n'):
    """Custom print function that writes to a specified file."""
    with open(file_path, 'a') as f:
        print(*args, file=f, sep=sep, end=end)

In [3]:
"""
TokenTree

Main class that can construct token trees, find the verified sequence against an
LLM and find statistics of the tree
"""
class TokenTree():
    def __init__(self, root_token=0):
        self.root = root_token
        self.children = []

    def get_token(self):
        return self.root

    def add_child(self, token):
        self.children.append(TokenTree(token))

    def delete_child(self, token):
        for i, child in enumerate(self.children):
            if child.get_token() == token:
                self.children.pop(i)

    """
    construct_from_chains

    Takes list of completions, where each completion is a list of token ids.
    Constructs a merged token tree

    Params:
        chains: list of list of tokens. Each list of tokens is a completion by an SSM
    """
    def construct_from_chains(self, chains):
        chains = [chain for chain in chains if len(chain) >= 1]
        level_tokens = [chain[0] for chain in chains]
        unique_child_tokens = list(set(level_tokens))

        for child_token in unique_child_tokens:
            self.add_child(child_token)
            child_tree = self.children[-1]
            child_chains = [chain[1:] for i, chain in enumerate(chains) if chain[0] == child_token]
            child_tree.construct_from_chains(child_chains)

    def display_tree(self, level=0):
        # DFS
        if level == 0:
            log_print("Displaying tree:\n")
        log_print("\t"*level + f"{self.get_token()}")
        for child_tree in self.children:
            child_tree.display_tree(level+1)

    """
    longest_matching_path

    Given a token path (generated by LLM), compare against tree and find longest
    prefix of the path that exists in the tree. This corresponds to tokens
    'verified' by the LLM

    Params:
        token_path: list of token ids (generated by LLM)
    """
    def longest_matching_path(self, token_path):
        if len(token_path) > 0:
            curr_token = token_path[0]
            for child in self.children:
                if child.get_token() == curr_token:
                    return [curr_token] + child.longest_matching_path(token_path[1:])
            return []
        else:
            return []

    """
    node_count

    Returns number of nodes in tree, including root
    """
    def node_count(self):
        nc = 1
        for child in self.children:
            nc += child.node_count()
        return nc

    """
    height_sum

    Returns sum of heights (depths actually) of all nodes, assuming the root
    is at height 0 and its children are at height 1 and so on
    """
    def height_sum(self):
        hs = 0
        for child in self.children:
            hs += child.height_sum() + child.node_count()
        return hs

    def average_height(self):
        return self.height_sum()/self.node_count()

    """
    abs_dev_sum

    Balance metric for the tree.
    Returns the average absolute deviation of node heights from the average height

    Params
        avg_height: If None, it will internally recompute. None should be used
                    only when calling at the root
        curr_height: Height of the root
    """
    def abs_dev_sum(self, avg_height=None, curr_height=0):
        if avg_height is None:
            avg_height = self.average_height()

        score = abs(curr_height-avg_height)
        for child in self.children:
            score += child.abs_dev_sum(avg_height, curr_height+1)

        return score

    def avg_abs_dev_sum(self):
        return self.abs_dev_sum()/self.node_count()

    def balance_score(self):
        return self.avg_abs_dev_sum()

In [4]:
with open(DATASET_PATH, 'r') as file:
    dataset = json.load(file)
print(len(dataset))

1000


In [5]:
# prompts = ["Hey, are you conscious?"]
prompts = dataset

for curr_prompt in tqdm(prompts):
    prompt = curr_prompt
    # Generate with LLM 
    llm_inputs = llm_tokenizer(prompt, return_tensors="pt")
    llm_inputs.to(DEVICE)
    input_token_length = len(llm_inputs.input_ids[0])
    len_total = input_token_length
    len_here = int(1e6)
    llm_generate_ids = llm_model.generate(llm_inputs.input_ids, max_new_tokens=MAX_GENERATION_LEN + TREE_DEPTH).clone().detach().cpu().numpy()
    llm_res = llm_tokenizer.batch_decode(llm_generate_ids, skip_special_tokens=True)[0]


    # Keep doing SSM speculation till max generation len is reached or llm finishes generating
    while len_total < len(llm_generate_ids[0]) :
        # Generate SSM completions
        ssm_gen_ids_lst = []
        for i in range(len(ssm_models)):
            ssm_inputs = ssm_tokenizers[i](prompt, return_tensors="pt").to(DEVICE)
            ssm_generate_ids = ssm_models[i].generate(ssm_inputs.input_ids, max_new_tokens=TREE_DEPTH, generation_config=ssm_generation_configs[i]).clone().detach().cpu().numpy()
            ssm_gen_ids_lst.append(ssm_generate_ids[0,-TREE_DEPTH:])
            

        # Find longest path through SSM tree that coincides with LLM (number of verified tokens)
        tt = TokenTree()
        tt.construct_from_chains(ssm_gen_ids_lst)
        tt.display_tree()
        lmp = tt.longest_matching_path(llm_generate_ids[0, len_total: len_total + TREE_DEPTH])
        log_print("Tree balance score:", tt.balance_score(), file_path=LOG_PATH)
        
        # Record the statistics
        len_here = 0 if not lmp else len(lmp)
        log_print("longest matching path, length:", len_here, file_path=LOG_PATH)
        log_print(tt.balance_score(), len_here, file_path=STATISTIC_PATH)

        # if there's no verified token, append next token
        if not lmp:
            lmp = llm_generate_ids[0, len_total]
        len_total += max(len_here, 1)

        llm_decoded_str = llm_tokenizer.batch_decode(torch.tensor([lmp], dtype=int), skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
        log_print("longest matching path decoded:", llm_decoded_str, file_path=LOG_PATH)
        prompt += llm_decoded_str
        log_print("current prompt: ", prompt, file_path=LOG_PATH)

    log_print("generated text for this prompt:", prompt, file_path=LOG_PATH)

  1%|          | 6/1000 [04:19<13:02:11, 47.21s/it]